<a href="https://colab.research.google.com/github/akankshakusf/Project-DeepLearning-SentimentAnalysis-DebertaHFT/blob/master/Sentiment_Analysis_with_BERT_in_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [12]:
!pip install transformers datasets

# Imports

In [13]:
#general package imports
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import cv2
import seaborn as sns
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
# import gensim.downloader as api
from PIL import Image

#tensorflow imports
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense, Flatten,InputLayer, BatchNormalization,Dropout, Input, LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,
                          DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          TFBertModel,create_optimizer)

In [33]:
BATCH_SIZE=8

# Data Preparation for Bert Model

In [14]:
dataset_id = 'imdb'
dataset = load_dataset(dataset_id)

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [16]:
dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## Bert Model

In [17]:
model_id = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_id)

In [18]:
tokenizer.is_fast

True

In [19]:
test_input_1='The weather of today is Great! zwp'
test_input_2='How are you doing?'
inputs = [test_input_1, test_input_2]
tokenizer.tokenize(inputs,)

['the',
 'weather',
 'of',
 'today',
 'is',
 'great',
 '!',
 'z',
 '##w',
 '##p',
 'how',
 'are',
 'you',
 'doing',
 '?']

In [20]:
output=tokenizer(inputs, padding=True, truncation=True,max_length=128)
print(output)

{'input_ids': [[101, 1996, 4633, 1997, 2651, 2003, 2307, 999, 1062, 2860, 2361, 102], [101, 2129, 2024, 2017, 2725, 1029, 102, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]}


In [21]:
tokenizer.decode(output['input_ids'][0])

'[CLS] the weather of today is great! zwp [SEP]'

In [22]:
tokenizer.decode(output['input_ids'][1])


'[CLS] how are you doing? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [23]:
def preprocess_function(examples):
  return tokenizer(examples["text"], padding=True,truncation=True,)

In [25]:
tokenized_dataset=dataset.map(preprocess_function,batched=True)

In [26]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [29]:
#tokenized_dataset['train'][0]

In [32]:
#data_collector = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
hf_train_dataset = tokenized_dataset['train'].to_tf_dataset(
      columns= ['input_ids','token_type_ids','attention_mask','label'],
      shuffle=True,
      batch_size=BATCH_SIZE,
)